안녕하세요.  
나만의 챗봇 만들기를 발표할 이연준입니다.

2022년 말 인공지능의 특이점이라고 불릴 수 있는 "챗GPT"가 등장하였습니다.  
챗GPT는 학계, 산업 등 다양한 분야에서 놀라운 기록을 세우고 있습니다.  
챗GPT의 등장으로 인공지능 언어모델을 이해하고 활용하는 능력이 정말 중요해 졌습니다.

이와 같은 배경에서 본 프로젝트는 인공지능 언어모델의 작동원리를 이해하기 위하여 다음과 같은 프로젝트를 준비하였습니다.  

먼저, 본 프로젝트에서는 챗GPT의 시작이되는 트렌스포머모델로 일상대화가 가능한 수준의 챗봇을 만들고, '나만의 챗봇'이기 때문에 챗봇 스스로가 자신을 "이연준의 챗봇"이라고 인식하게 하는 것이 목표입니다.  
챗봇이 학습할 데이터에 "이연준의 챗봇"이라는 데이터를 학습시킴으로써 챗봇 스스로가 "이연준의 챗봇"이라 인식할 수 있을 것으로 기대 됩니다.

1. 데이터 준비  

나만의 챗봇을 만들기 위하여 "감성대화 말뭉치" 데이터를 사용하겠습니다.  

감성대화 말뭉치는 AI 언어모델을 만들기 위하여 수집된 데이터입니다.  
5개월이 넘는 기간동안 일반인 1500명을 대상으로 수집되었고, 60가지의 감정 상태가 포함되어있습니다.  
감성대화 말뭉치는 인공지능 언어모델을 개발하기 편리하도록 잘 정제되어있어서, 일상대화를 할 수 있는 챗봇을 만들기 위한 적절한 데이터라고 여겨집니다.  

강성대화 말뭉치 데이터 안에는 총 58,268개 말뭉치 데이터가 있습니다.  
이때 하나의 말뭉치 데이터는 질문과 답변이 3쌍으로 이루어진 데이터입니다.  
본 프로젝트에서는 간단한 일상대화를 하는 챗봇을 만들 것이기 때문에 1턴 데이터만 사용하도록 하겠습니다.  

본 프로젝트에서는 사용하기 편리한 서브워드텍스트인코더를 활용하여 토큰화와 벡터화를 진행하였습니다.  
토큰화는 다음과 같이 문장으로 이루어진 데이터를 작게 자르는 것이고, 벡터화는 텍스트를 숫자로 변환하는 것입니다.  

본 프로젝트에서 사용할 트랜스포머 모델은 3가지 종류의 데이터가 필요합니다.  
질문 데이터는 START 토큰과 END 토큰이 포함되어 최초 인코더에 입력됩니다.  
답변 데이터는 두가지로 구분되는데, START 토큰이 있는 답변 데이터는 디코더에 입력되고, END 토큰만 있는 답변 데이터는 출력타겟으로 사용됩니다.  



2. 모델 구성  

본 프로젝트에서는 트랜스포머 모델로 챗봇을 만듧니다.  
모델 구성은 Attention Is All You Need 논문을 배경으로 TensorFlow의 라이브러리를 적극 활용하였습니다.  

트랜스포머 모델은 위치인코딩이 포함된 임베딩으로 시작합니다.  
이미 토큰화, 벡터화를 거친 데이터에 대하여 임베딩벡터로 변환하고, 변환된 임베딩 벡터에 사인, 코사인 함수를 활용하여 위치정보를 생성합니다.  
생성된 위치 정보를 시각화하면 다음과 같습니다.  
가로축에는 본 프로젝트에서 사용하는 약10,000개의 단어집합, 세로축에는 입베딩벡터의 차원에 대하여 -1에서 1까지의 값이 할당되었습니다.  

인코더의 첫번째 층은 글로벌 셀프 어텐션 레이어입니다.  
이곳에는 입력된 데이터에 대하여 쿼리, 키, 벨류 연산을 수행하고, 입력값과 출력값을 합치고 정규화하는 과정을 거칩니다.  


디코더의 첫번째 층은 캐주얼 셀프 어텐션 레이어입니다.  
이곳에서도 역시 입력된 데이터에 대하여 뭐리, 키, 벨류 연산을 수행하는데, 단 답변에 대한 예측을 위하여 마스킹 과정이 포함 됩니다.  

다음으로 인코더의 출력은 크로스 어텐션 레이어에서 합쳐지게 됩니다.  
이곳에서도 역시 쿼리, 키, 벨류 연산이 수행되는데, 이때 쿼리는 디코더의 입력 데이터이고, 키, 벨류는 인코더의 출력데이터입니다.  

학습 데이터의 질문과 답변 데이터를 통하여 각 단어의 출현 확률을 추정하고 학습합니다.  
그리고 [END]단어가 추정 될 때까지 단어를 예측하게 됩니다.  


3. 모델 학습 및 평가  

모델 학습을 위한 몇가지 하이퍼파라미터를 지정합니다.  
모델의 학습률은 학습진행에 따라 학습률이 조정되는 메커니즘을 사용하였습니다.  
모델을 경량화하여 레이어와 임베딩차원 등을 지정하였고, 과정합방지를 위한 Drop Out를 적용하였습니다.  

모델 학습을 진행한 결과 4 에포크가 넘어가면 과적합이 발생하고, 오히려 검증데이터의 손실이 늘어나는 것을 알 수 있습니다.  

학습된 모델을 평가하기 위하여 몇가지 질문을 하였습니다.  
첫번째 "취업하기 위해서 열심히 준비하고 있지만, 직장을 구하지 못할까봐 걱정되"라고 입력하면, "취업 준비로 걱정이 많으시군요."라고 적절히 답변하였습니다.  
두번째, "딸이 나에게 선물을 줬어. 그래서 너무 기뻐"라고 질문하면, "딸이 선물을 줘서 기쁘시군요."라고 답변하였습니다.  
세번째, "너는 누구니?"라고 질문하였더니, 학습한 대로 "저는 이연준님의 챗봇 이라고 합니다."라고 대답하였습니다.  
네번재, "아무리 생각해도 너는 멍청한 것 같아."라고 질문하였더니, "일종의 애쓰는 것으로 인해 열등감을 느끼시는 군요."라고 답변하였습니다.  
